In [1]:
# -*- coding: utf-8 -*-
# 选项 3: 图片转摘要检索 + 原图生成 RAG 实现示例
# 使用 MLLM (占位符) 生成图片摘要，使用文本嵌入模型索引原文+摘要，
# 检索后获取原文+原图，使用 MLLM (占位符) 生成

import sqlite3
import os
import numpy as np
from typing import List, Dict, Union, Optional, Tuple
import faiss
from transformers import AutoTokenizer, AutoModel # 用于文本嵌入
from PIL import Image
import torch
import zhipuai # 用于占位符 MLLM 生成
import json
import time
import random

# --- 数据加载与图片关联函数 (与原示例相同) ---
def load_data_from_json_and_associate_images(json_path: str, image_dir: str) -> List[Dict]:
    """
    # 从 JSON 文件加载文档数据，并尝试在指定的图像目录中关联对应的图片文件。
    # (与原示例代码相同，此处省略详细注释)
    """
    if not os.path.exists(json_path):
        print(f"错误: 未找到 JSON 文件 '{json_path}'。")
        return []
    documents = []
    image_extensions = ['.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff']
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            json_data = json.load(f)
    except Exception as e:
        print(f"错误: 读取或解析 JSON 文件 '{json_path}' 失败: {e}")
        return []
    for item in json_data:
        doc_id = item.get('name')
        text_content = item.get('description')
        if not doc_id or not text_content:
            continue
        image_path = None
        if image_dir and os.path.exists(image_dir):
             for ext in image_extensions:
                 potential_image_path = os.path.join(image_dir, str(doc_id) + ext)
                 if os.path.exists(potential_image_path):
                     image_path = potential_image_path
                     break
        documents.append({
            'id': str(doc_id),
            'text': str(text_content) if text_content is not None else None,
            'image_path': image_path
        })
    print(f"已加载 {len(json_data)} 条记录，成功准备 {len(documents)} 个文档。")
    return documents

# --- 图片摘要生成器 (占位符) ---
# 与选项 2 相同
class ImageSummarizer_Placeholder:
    """
    # 图片摘要生成器 (占位符):
    # !!! 实际应用中需要调用真正的 MLLM API 或模型 !!!
    """
    def __init__(self, model_identifier="PlaceholderSummarizer"):
        print(f"ImageSummarizer (占位符) 初始化。模型标识: {model_identifier}")
    def summarize(self, image_path: str) -> Optional[str]:
        if not image_path or not os.path.exists(image_path): return None
        try:
            # --- !!! 在此替换为调用 MLLM 生成摘要的逻辑 !!! ---
            filename = os.path.basename(image_path)
            summary = f"[图像摘要占位符 Option3] 这是对图像 '{filename}' 的描述，用于文本检索。原始图像将在生成阶段使用。"
            print(f"  - [摘要器占位符 Opt3] 为 '{filename}' 生成了摘要。")
            return summary
        except Exception as e: print(f"  - [摘要器占位符 Opt3] 生成摘要出错: {e}"); return None

# --- 文本编码器 (TextEncoder) ---
# 与选项 2 相同
class TextEncoder:
    """
    # 使用 Hugging Face Transformers 加载文本嵌入模型。
    """
    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2", device: Optional[str] = None):
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(model_name)
            self.model = AutoModel.from_pretrained(model_name)
            self.vector_dimension = self.model.config.hidden_size
            if device: self.device = torch.device(device)
            elif torch.cuda.is_available(): self.device = torch.device("cuda")
            else: self.device = torch.device("cpu")
            self.model.to(self.device); self.model.eval()
            print(f"TextEncoder 初始化成功，模型: {model_name}, 维度: {self.vector_dimension}, 设备: {self.device}")
        except Exception as e: print(f"加载文本嵌入模型 {model_name} 失败: {e}"); raise e

    def encode(self, text: str) -> Optional[np.ndarray]:
        if not text: return None
        try:
            with torch.no_grad():
                inputs = self.tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=512).to(self.device)
                outputs = self.model(**inputs)
                attention_mask = inputs['attention_mask']
                last_hidden_state = outputs.last_hidden_state
                mean_pooled_vector = torch.sum(last_hidden_state * attention_mask.unsqueeze(-1), dim=1) / torch.clamp(torch.sum(attention_mask, dim=1, keepdim=True), min=1e-9)
                vector = mean_pooled_vector.squeeze().cpu().numpy().astype('float32')
                norm = np.linalg.norm(vector)
                return vector / norm if norm > 1e-6 else np.zeros_like(vector)
        except Exception as e: print(f"编码文本 '{text[:30]}...' 出错: {e}"); return None

# --- 2. 索引器 (Indexer) - 选项 3 ---
# 为图像生成摘要，使用文本编码器索引原文+摘要，存储原文+原图路径
class Indexer_Option3:
    """
    # 索引器 (选项3):
    # - 使用 ImageSummarizer 为图像生成文本摘要 (仅用于检索)。
    # - 使用 TextEncoder 分别编码原始文本块和生成的图像摘要。
    # - 将所有这些 *文本向量* 存储到 Faiss 索引中。
    # - 将原始文本块和 *原始图像路径* 存储到 SQLite 数据库中。
    # - 需要映射关系：向量 ID -> (原始文本 或 原始图像路径)
    """
    def __init__(self, db_path: str, faiss_index_path: str, text_embed_model: str, image_summarizer: ImageSummarizer_Placeholder):
        self.db_path = db_path
        self.faiss_index_path = faiss_index_path
        self.encoder = TextEncoder(text_embed_model) # 文本编码器
        self.summarizer = image_summarizer # 图片摘要器
        self.vector_dimension = self.encoder.vector_dimension

        self._init_db()
        self._load_or_create_faiss_index()

    def _init_db(self):
        """初始化 SQLite 数据库 (存储原文文本和原图路径)"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                # vector_index_id: Faiss ID, 主键
                # doc_id: 原始文档 ID
                # content_source: 'original_text' 或 'image_summary' (指明这个向量代表什么)
                # text_content: 存储原始文本 (如果是文本源)
                # image_path: 存储原始图像路径 (如果是图像摘要源，或文本也关联图像时)
                cursor.execute('''
                    CREATE TABLE IF NOT EXISTS items_with_raw_image (
                        vector_index_id INTEGER PRIMARY KEY,
                        doc_id TEXT NOT NULL,
                        content_source TEXT NOT NULL CHECK(content_source IN ('original_text', 'image_summary')),
                        text_content TEXT,
                        image_path TEXT
                    )
                ''')
                cursor.execute("CREATE INDEX IF NOT EXISTS idx_doc_id_source_opt3 ON items_with_raw_image (doc_id, content_source)")
                conn.commit()
                print(f"数据库已初始化或连接成功: {self.db_path}")
        except Exception as e: print(f"初始化数据库失败: {e}"); raise e

    def _load_or_create_faiss_index(self):
        """加载或创建 Faiss 向量索引 (只存文本向量)"""
        try:
            if os.path.exists(self.faiss_index_path):
                self.index = faiss.read_index(self.faiss_index_path)
                print(f"成功加载 Faiss 索引: {self.faiss_index_path}, 包含 {self.index.ntotal} 个向量。")
            else:
                print("未找到 Faiss 索引文件，创建新的空索引。")
                quantizer = faiss.IndexFlatIP(self.vector_dimension)
                self.index = faiss.IndexIDMap2(quantizer)
                print(f"创建了新的 Faiss IndexIDMap2 (内积) 索引 (维度: {self.vector_dimension})。")
        except Exception as e:
            print(f"加载或创建 Faiss 索引失败: {e}。将创建一个新的空索引。")
            quantizer = faiss.IndexFlatIP(self.vector_dimension)
            self.index = faiss.IndexIDMap2(quantizer)

    def index_documents(self, documents: List[Dict]):
        """对文档列表进行索引"""
        if not documents: return
        print(f"开始处理 {len(documents)} 个文档进行索引 (选项 3: 摘要检索+原图生成)...")

        vectors_to_add = []
        vector_ids_for_batch = []
        data_to_store = [] # (vec_id, doc_id, source, text, img_path)

        start_vector_index_id = self.index.ntotal
        next_vector_index_id = start_vector_index_id

        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()

        processed_count, skipped_duplicates, skipped_encoding_errors, summary_errors = 0, 0, 0, 0

        for doc in documents:
            doc_id = doc.get('id')
            text = doc.get('text')
            image_path = doc.get('image_path') # 原始图片路径

            if not doc_id: continue

            # --- 1. 索引原始文本部分 ---
            if text:
                cursor.execute("SELECT 1 FROM items_with_raw_image WHERE doc_id = ? AND content_source = 'original_text'", (doc_id,))
                if cursor.fetchone(): skipped_duplicates += 1
                else:
                    text_vector = self.encoder.encode(text)
                    if text_vector is not None:
                        vectors_to_add.append(text_vector)
                        vector_ids_for_batch.append(next_vector_index_id)
                        # 存储原始文本，并关联原始图片路径（如果存在）
                        data_to_store.append((next_vector_index_id, doc_id, 'original_text', text, image_path if image_path and os.path.exists(image_path) else None))
                        next_vector_index_id += 1
                    else: skipped_encoding_errors += 1

            # --- 2. 索引图像摘要部分 ---
            if image_path and os.path.exists(image_path):
                cursor.execute("SELECT 1 FROM items_with_raw_image WHERE doc_id = ? AND content_source = 'image_summary'", (doc_id,))
                if cursor.fetchone(): skipped_duplicates += 1
                else:
                    summary_text = self.summarizer.summarize(image_path)
                    if summary_text:
                        summary_vector = self.encoder.encode(summary_text)
                        if summary_vector is not None:
                            vectors_to_add.append(summary_vector)
                            vector_ids_for_batch.append(next_vector_index_id)
                            # 存储标记为摘要源，不存摘要文本，但存储 *原始图片路径*
                            data_to_store.append((next_vector_index_id, doc_id, 'image_summary', None, image_path))
                            next_vector_index_id += 1
                        else: skipped_encoding_errors += 1
                    else: summary_errors += 1
            elif image_path: pass # 文件不存在

            processed_count += 1
            if processed_count % 50 == 0: print(f"  已处理 {processed_count}/{len(documents)}...")

        # --- 批量添加到 Faiss 和 SQLite ---
        if vectors_to_add:
            try:
                vectors_np = np.array(vectors_to_add, dtype='float32')
                ids_np = np.array(vector_ids_for_batch, dtype='int64')
                self.index.add_with_ids(vectors_np, ids_np)
                print(f"成功向 Faiss 添加 {len(vectors_np)} 个新文本向量。当前总数: {self.index.ntotal}")

                cursor.executemany(
                    "INSERT INTO items_with_raw_image (vector_index_id, doc_id, content_source, text_content, image_path) VALUES (?, ?, ?, ?, ?)",
                    data_to_store
                )
                conn.commit()
                print(f"成功向数据库存储 {len(data_to_store)} 条新记录 (关联原文或原图)。")
            except Exception as e: print(f"批量添加或存储时出错: {e}"); conn.rollback()
        else: print("没有新的有效向量或数据需要添加。")

        conn.close()
        print(f"索引过程完成。总处理: {processed_count}, 跳过重复: {skipped_duplicates}, 跳过编码错误: {skipped_encoding_errors}, 图片摘要失败: {summary_errors}")

    def get_item_for_generation(self, vector_index_id: int) -> Optional[Dict]:
        """根据 vector_index_id 获取用于 *生成* 的原始数据 (文本或图片路径)"""
        try:
            with sqlite3.connect(self.db_path) as conn:
                cursor = conn.cursor()
                # 获取 vector_index_id 对应的记录
                cursor.execute(
                    "SELECT doc_id, content_source, text_content, image_path FROM items_with_raw_image WHERE vector_index_id = ?",
                    (vector_index_id,)
                )
                row = cursor.fetchone()
                if row:
                    doc_id, source, text, img_path = row
                    # 返回用于生成的数据
                    return {
                        'vector_index_id': vector_index_id, # 用于调试或关联
                        'doc_id': doc_id,
                        'source_of_hit': source, # 这个向量是代表文本还是图片摘要
                        'text_to_provide': text, # 如果是文本源，提供文本
                        'image_path_to_provide': img_path # 提供关联的原始图片路径 (无论源是文本还是摘要)
                    }
                return None
        except Exception as e:
             print(f"从数据库根据 vector_index_id {vector_index_id} 获取生成数据出错: {e}")
             return None

    def get_index_count(self) -> int: return self.index.ntotal if hasattr(self, 'index') else 0
    def get_db_item_count(self) -> int:
         try:
             with sqlite3.connect(self.db_path) as conn: cursor = conn.cursor(); cursor.execute("SELECT COUNT(*) FROM items_with_raw_image"); return cursor.fetchone()[0]
         except Exception: return 0
    def save_index(self):
        if hasattr(self, 'index') and self.index.ntotal > 0:
            try: faiss.write_index(self.index, self.faiss_index_path); print(f"Faiss 索引已保存: {self.faiss_index_path}")
            except Exception as e: print(f"保存 Faiss 索引失败: {e}")
        elif hasattr(self, 'index'): print("索引为空，跳过保存。")
        else: print("索引未初始化，跳过保存。")
    def close(self): self.save_index()

# --- 3. 检索器 (Retriever) - 选项 3 ---
# 使用文本编码器对文本查询编码，在 Faiss 中搜索文本向量，获取原文+原图路径
class Retriever_Option3:
    """
    # 检索器 (选项3):
    # - 使用 Indexer 的 TextEncoder 对用户的 *纯文本查询* 进行编码。
    # - 在 Faiss 索引（只包含文本向量）中搜索最相似的文本向量。
    # - 根据返回的 vector_index_id 从数据库获取对应的 *原始文本* 和 *原始图像路径*。
    """
    def __init__(self, indexer: Indexer_Option3):
        if not isinstance(indexer, Indexer_Option3): raise ValueError("必须提供有效的 Indexer_Option3 实例。")
        self.indexer = indexer
        self.encoder = indexer.encoder # TextEncoder
        self.index = indexer.index
        if self.index is None or self.index.ntotal == 0: print("警告: 检索器初始化时发现索引为空。")

    def retrieve(self, query: str, k: int = 5) -> List[Dict]:
        """
        # 执行检索流程 (仅限文本查询)，返回用于生成的原始图文数据。

        Args:
            query: 用户纯文本查询。
            k: 检索数量。

        Returns:
            包含原始文本、原始图像路径和得分的字典列表。
            [{'doc_id': ..., 'text_to_provide': ..., 'image_path_to_provide': ..., 'score': ...}, ...]
        """
        if self.index is None or self.index.ntotal == 0: print("错误: 向量索引不可用或为空。"); return []
        if not isinstance(query, str) or not query: print("错误: 查询必须是非空文本字符串。"); return []

        # 1. 查询编码 (纯文本)
        query_vector = self.encoder.encode(query)
        if query_vector is None: print("错误: 查询编码失败。"); return []
        query_vector = query_vector.reshape(1, self.indexer.vector_dimension)

        # 2. Faiss 搜索 (文本向量)
        try:
            print(f"  - 正在 Faiss 索引中搜索 Top {k} 文本向量...")
            scores, vector_index_ids = self.index.search(query_vector, k)
            print(f"  - Faiss 搜索完成，找到 {len(vector_index_ids[0])} 个结果。")
        except Exception as e: print(f"  - Faiss 搜索失败: {e}"); return []

        # 3. 获取用于生成的原始数据 (原文+原图路径)
        retrieved_items_for_generation = []
        print("  - 正在从数据库获取用于生成的原始图文信息...")
        retrieved_doc_ids = set() # 用于对相同 doc_id 的结果进行合并或去重 (简单策略：只保留最高分)
        temp_results = {} # {doc_id: best_item_data}

        for i, vec_id in enumerate(vector_index_ids[0]):
            if vec_id == -1: continue
            # 使用 get_item_for_generation 获取生成所需数据
            item_data = self.indexer.get_item_for_generation(int(vec_id))
            if item_data:
                item_data['score'] = float(scores[0][i])
                doc_id = item_data['doc_id']
                # 如果这个 doc_id 还没遇到过，或者当前得分更高，则更新
                if doc_id not in temp_results or item_data['score'] > temp_results[doc_id]['score']:
                     temp_results[doc_id] = item_data
            else: print(f"  - 警告: 未找到 vector_index_id {vec_id} 对应的数据库记录。")

        # 将去重/合并后的结果放入最终列表
        retrieved_items_for_generation = list(temp_results.values())
        # 按分数重新排序 (因为合并可能打乱顺序)
        retrieved_items_for_generation.sort(key=lambda x: x['score'], reverse=True)

        print(f"  - 成功获取 {len(retrieved_items_for_generation)} 个唯一文档的图文数据用于生成。")
        return retrieved_items_for_generation

    def close(self): pass

# --- 4. 生成器 (Generator) - 选项 3 (使用 MLLM - 占位符) ---
# 与选项 1 的 Generator 类似，需要处理图文混合输入
class Generator_Option3:
    """
    # 生成器 (选项3):
    # - **需要使用真正的多模态大语言模型 (MLLM)。**
    # - 接收检索到的原始文本和原始图像路径。
    # - 构建能被 MLLM API 接受的 Prompt (含文本和图像)。
    # - 调用 MLLM API 生成答案。
    #
    # !!! 注意: 使用 ZhipuAI 文本模型作为占位符 !!!
    # !!! 需要替换为实际 MLLM 的 API 调用和 Prompt 构建 !!!
    """
    def __init__(self, api_key: Optional[str] = None, model_name: str = "glm-4v",
                 fallback_text_model: str = "glm-4-flash"):
        final_api_key = api_key if api_key else os.getenv("ZHIPUAI_API_KEY")
        if not final_api_key: raise ValueError("未提供 ZHIPUAI_API_KEY。")
        try:
            self.client = zhipuai.ZhipuAI(api_key=final_api_key)
            self.model_name = model_name
            self.fallback_model = fallback_text_model
            print(f"Generator (选项 3 - MLLM 占位符) 初始化成功。目标模型: {self.model_name}, 备用: {self.fallback_model}")
        except Exception as e: print(f"初始化智谱 AI 客户端失败: {e}"); raise e

    def generate(self, query: str, context: List[Dict]) -> str:
        """
        # 接收文本查询和包含原始文本/图像路径的上下文，生成答案。

        Args:
            query: 原始用户文本查询。
            context: 从 Retriever_Option3 获取的列表。
                     [{'doc_id':..., 'text_to_provide':..., 'image_path_to_provide':..., 'score':...}, ...]

        Returns:
            MLLM (或占位符 LLM) 生成的文本响应。
        """
        print(f"正在使用 {len(context)} 个检索到的图文条目为查询生成响应...")

        # --- !!! MLLM Prompt 构建逻辑 (需要根据实际 MLLM API 调整) !!! ---
        query_text = query

        context_texts = []
        context_images = [] # 存储图像路径等信息
        for item in context:
            # 添加文本上下文
            if item['text_to_provide']:
                truncated_text = item['text_to_provide'][:500] + ('...' if len(item['text_to_provide']) > 500 else '')
                context_texts.append(f"- [来源 Doc ID: {item['doc_id']}, 得分: {item['score']:.4f}] {truncated_text}")
            # 收集需要展示给 MLLM 的图像信息
            if item['image_path_to_provide'] and os.path.exists(item['image_path_to_provide']):
                 # 避免重复添加同一张图片
                 if not any(img['path'] == item['image_path_to_provide'] for img in context_images):
                      context_images.append({
                          "path": item['image_path_to_provide'],
                          "doc_id": item['doc_id'], # 可以是关联的文档 ID
                          "score": item['score'] # 可以是关联的最高分
                      })

        # 构建 MLLM 需要的 messages 结构 (同选项 1 的占位符逻辑)
        messages = []
        system_prompt = f"""
        你是一个多模态助手，结合文本和图像信息回答问题。请根据用户查询和下面提供的文本及图像信息作答。
        严格根据提供的信息，信息不足请说明。
        参考文本信息:
        {chr(10).join(context_texts) if context_texts else '无相关文本信息。'}
        """.strip()
        messages.append({"role": "system", "content": system_prompt})

        user_content_parts = [{"type": "text", "text": query_text}]
        if context_images:
            user_content_parts.append({"type": "text", "text": "\n请参考以下图像:"})
            image_count = 0
            for img_info in context_images:
                # --- !!! 此处需要将图像转换为 MLLM API 格式 !!! ---
                # (占位符：只添加文本描述)
                user_content_parts.append({
                    "type": "text",
                    "text": f"\n- 图像 {image_count+1} (关联 Doc ID: {img_info['doc_id']}, 路径: {img_info['path']}, 最高分: {img_info['score']:.4f}) [图像内容需由 MLLM 查看]"
                })
                image_count += 1

        messages.append({"role": "user", "content": user_content_parts})

        print("  - [占位符] MLLM Prompt 构建完成 (需要适配实际 MLLM API 格式)。")

        # --- !!! 调用 MLLM API (占位符逻辑) !!! ---
        print(f"  - [占位符] 正在尝试调用目标 MLLM '{self.model_name}'...")
        try:
            # --- !!! 此处应为实际 MLLM API 调用 !!! ---
            # response = self.client.chat.completions.create(model=self.model_name, messages=messages, ...)

            # --- 回退到文本模型进行演示 ---
            print(f"  - [占位符] 无法直接调用 MLLM，将使用备用文本模型 '{self.fallback_model}' 处理纯文本部分...")
            text_only_messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": query_text + "\n\n(注意: 我无法直接看到图片，请根据提供的文本描述回答。)"}
            ]
            response = self.client.chat.completions.create(
                model=self.fallback_model, messages=text_only_messages, temperature=0.7, max_tokens=1024
            )
            llm_response = response.choices[0].message.content
            print(f"  - [占位符] 使用备用文本模型调用成功。")

        except Exception as e:
            print(f"  - 调用 MLLM (或备用模型) 时出错: {e}")
            return f"调用 LLM 出错: {e}"

        # 后处理
        processed_response = llm_response.strip()
        print("  - 响应生成和后处理完成。")
        return processed_response

# --- 选项 3 示例使用流程 ---
if __name__ == "__main__":
    print("\n" + "="*10 + " 选项 3: 摘要检索+原图生成 RAG 示例 " + "="*10 + "\n")

    # --- 配置 ---
    json_data_path = 'data.json'
    image_directory_path = 'images'
    db_file_opt3 = 'rag_option3.db'
    faiss_index_file_opt3 = 'rag_option3.faiss'
    TEXT_EMBED_MODEL_OPT3 = "sentence-transformers/all-MiniLM-L6-v2"
    # !! 选择你的目标 MLLM 模型名称 !!
    TARGET_MLLM_OPT3 = "glm-4v"
    FALLBACK_TEXT_LLM_OPT3 = "glm-4-flash"

    # --- 清理 ---
    print("--- 清理旧文件 ---")
    if os.path.exists(db_file_opt3): os.remove(db_file_opt3)
    if os.path.exists(faiss_index_file_opt3): os.remove(faiss_index_file_opt3)
    print("--- 清理完成 ---")

    # --- 1. 加载数据 ---
    print("\n--- 步骤 1: 加载数据 ---")
    documents = load_data_from_json_and_associate_images(json_data_path, image_directory_path)
    if not documents: exit("错误: 未加载到文档。")
    print("--- 步骤 1 完成 ---")

    # --- 1.5 初始化图片摘要器 (占位符) ---
    print("\n--- 步骤 1.5: 初始化图片摘要器 (占位符) ---")
    image_summarizer_opt3 = ImageSummarizer_Placeholder()
    print("--- 步骤 1.5 完成 ---")

    # --- 2. 初始化 Indexer 并索引 ---
    print("\n--- 步骤 2: 初始化 Indexer (选项 3) 并索引 ---")
    indexer_opt3 = None
    try:
        indexer_opt3 = Indexer_Option3(
            db_path=db_file_opt3,
            faiss_index_path=faiss_index_file_opt3,
            text_embed_model=TEXT_EMBED_MODEL_OPT3,
            image_summarizer=image_summarizer_opt3
        )
        indexer_opt3.index_documents(documents)
        print(f"索引完成。向量总数: {indexer_opt3.get_index_count()}, 数据库条目数: {indexer_opt3.get_db_item_count()}")
        if indexer_opt3.get_index_count() == 0: print("错误: 索引为空！"); indexer_opt3 = None
    except Exception as e:
        print(f"Indexer 初始化或索引失败: {e}")
        indexer_opt3 = None
    print("--- 步骤 2 完成 ---")

    # --- 3. 初始化 Retriever ---
    print("\n--- 步骤 3: 初始化 Retriever (选项 3) ---")
    retriever_opt3 = None
    if indexer_opt3:
        try:
            retriever_opt3 = Retriever_Option3(indexer=indexer_opt3)
            print("Retriever 初始化成功。")
        except Exception as e: print(f"Retriever 初始化失败: {e}")
    else: print("Indexer 不可用，跳过 Retriever 初始化。")
    print("--- 步骤 3 完成 ---")

    # --- 4. 初始化 Generator ---
    print("\n--- 步骤 4: 初始化 Generator (选项 3 - MLLM 占位符) ---")
    generator_opt3 = None
    if os.getenv("ZHIPUAI_API_KEY"):
        try:
            generator_opt3 = Generator_Option3(model_name=TARGET_MLLM_OPT3, fallback_text_model=FALLBACK_TEXT_LLM_OPT3)
            print("Generator 初始化成功。")
        except Exception as e: print(f"Generator 初始化失败: {e}")
    else: print("ZHIPUAI_API_KEY 未设置，跳过 Generator 初始化。")
    print("--- 步骤 4 完成 ---")

    # --- 5. 执行查询 (仅文本) ---
    print("\n--- 步骤 5: 执行查询示例 (仅文本) ---")
    if retriever_opt3 and generator_opt3:
        # 辅助函数打印结果
        def print_retrieved_items_opt3(items: List[Dict]):
            if not items: print("    (未检索到条目)")
            for i, item in enumerate(items):
                score_str = f"得分: {item.get('score', 'N/A'):.4f}"
                print(f"    {i+1}. Doc ID: {item['doc_id']}, {score_str}")
                if item['text_to_provide']:
                    text_preview = item['text_to_provide'][:100] + ('...' if len(item['text_to_provide']) > 100 else '')
                    print(f"       关联文本: {text_preview}")
                if item['image_path_to_provide']:
                    print(f"       关联图像文件: {os.path.basename(item['image_path_to_provide'])}")
                print("    " + "-" * 15)

        # 查询示例 (纯文本)
        queries_opt3 = [
            "带隙基准电路的核心原理是什么？请结合相关电路图说明。", # 明确要求结合图
            "比较不同文档中关于 PTAT 电流生成的电路实现。",
        ]

        for i, query in enumerate(queries_opt3):
            print(f"\n--- 文本查询 {i+1}/{len(queries_opt3)} ---")
            print(f"查询: {query}")

            try:
                # Retriever 返回原始文本和原始图片路径
                retrieved_context = retriever_opt3.retrieve(query, k=5)
                print(f"  检索到 {len(retrieved_context)} 个用于生成的图文条目:")
                print_retrieved_items_opt3(retrieved_context)

                if retrieved_context:
                    print("\n  生成响应 (使用 MLLM 或占位符):")
                    # Generator 接收原始文本和图片路径，需要 MLLM 处理
                    response = generator_opt3.generate(query, retrieved_context)
                    print(response)
                else:
                    print("\n  未检索到上下文，无法生成响应。")
            except Exception as e:
                print(f"\n  执行查询或生成时出错: {e}")
            time.sleep(1)

    else:
        print("Retriever 或 Generator 未初始化，跳过查询执行。")
    print("--- 步骤 5 完成 ---")

    # --- 6. 清理 ---
    print("\n--- 步骤 6: 清理资源 ---")
    if indexer_opt3: indexer_opt3.close()
    # Retriever 和 Generator 无需关闭
    print("--- 清理完成 ---")
    print("\n" + "="*10 + " 选项 3 示例结束 " + "="*10 + "\n")

d:\ProgramData\anaconda3\envs\mrag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



========== 选项 3: 摘要检索+原图生成 RAG 示例 ==========

--- 清理旧文件 ---
--- 清理完成 ---

--- 步骤 1: 加载数据 ---
已加载 219 条记录，成功准备 219 个文档。
--- 步骤 1 完成 ---

--- 步骤 1.5: 初始化图片摘要器 (占位符) ---
ImageSummarizer (占位符) 初始化。模型标识: PlaceholderSummarizer
--- 步骤 1.5 完成 ---

--- 步骤 2: 初始化 Indexer (选项 3) 并索引 ---
TextEncoder 初始化成功，模型: sentence-transformers/all-MiniLM-L6-v2, 维度: 384, 设备: cpu
数据库已初始化或连接成功: rag_option3.db
未找到 Faiss 索引文件，创建新的空索引。
创建了新的 Faiss IndexIDMap2 (内积) 索引 (维度: 384)。
开始处理 219 个文档进行索引 (选项 3: 摘要检索+原图生成)...
  - [摘要器占位符 Opt3] 为 'Bandgap1.jpg' 生成了摘要。
  - [摘要器占位符 Opt3] 为 'Bandgap10.jpg' 生成了摘要。
  - [摘要器占位符 Opt3] 为 'Bandgap11.jpg' 生成了摘要。
  - [摘要器占位符 Opt3] 为 'Bandgap23.png' 生成了摘要。
  - [摘要器占位符 Opt3] 为 'Bandgap26.png' 生成了摘要。
  - [摘要器占位符 Opt3] 为 'Bandgap3.jpg' 生成了摘要。
  - [摘要器占位符 Opt3] 为 'Bandgap31.png' 生成了摘要。
  - [摘要器占位符 Opt3] 为 'Bandgap32.png' 生成了摘要。
  - [摘要器占位符 Opt3] 为 'Bandgap33.png' 生成了摘要。
  - [摘要器占位符 Opt3] 为 'Bandgap34.png' 生成了摘要。
  - [摘要器占位符 Opt3] 为 'Bandgap35.png' 生成了摘要。
  - [摘要器占位符 Opt3] 为 'Bandgap36.png' 生成